##### Goal: Write function using XGBoost to run model from start to finish

##### ***I am having environment issues, so not much has been done to this XGBoost repo yet***

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score, KFold #, GridSearchCV
from sklearn.metrics import mean_squared_error, confusion_matrix

# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import 

from xgboost import (
    XGBClassifier,
    XGBRegressor
)

In [2]:
df = pd.read_csv('../data/model_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9792 entries, 0 to 9791
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fighter         9792 non-null   object 
 1   opp_fighter     9792 non-null   object 
 2   title_bout      9792 non-null   int64  
 3   weight_class    9792 non-null   object 
 4   gender          9792 non-null   object 
 5   stance          9790 non-null   object 
 6   reach_cms       9792 non-null   float64
 7   height_cms      9792 non-null   float64
 8   weight_lbs      9792 non-null   int64  
 9   age             9792 non-null   int64  
 10  opp_stance      9790 non-null   object 
 11  opp_reach_cms   9792 non-null   float64
 12  opp_height_cms  9792 non-null   float64
 13  opp_weight_lbs  9792 non-null   int64  
 14  opp_age         9792 non-null   int64  
 15  winner          9792 non-null   int64  
dtypes: float64(4), int64(6), object(6)
memory usage: 1.2+ MB


In [3]:
features: list[str,...] = [
#     'fighter',
#     'opp_fighter',
    'title_bout',
#     'weight_class',
#     'gender',
#     'stance',
    'reach_cms',
    'height_cms',
    'weight_lbs',
    'age',
#     'opp_stance',
    'opp_reach_cms',
    'opp_height_cms',
    'opp_weight_lbs',
    'opp_age',
    'winner'
]

In [4]:
def run_model(df: pd.DataFrame, target: str, *args, **kwargs) -> None:
    
    features: list[str,...] = kwargs.get('features', [column for column in df.columns if column != target])
    
    X: pd.DataFrame = df[features]
    y: pd.Series = df[target]
    
#     X = X transformed
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=42)
    
    xgb = XGBClassifier()
    
    xgb.fit(X_train, y_train)
    
    
    # Output model results
    predictions = xgb.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)

    print(f'\nModel score for training data for {target}: {xgb.score(X_train, y_train):.4f}')
    print(f'Model score for testing data for {target}: {xgb.score(X_test, y_test):.4f}')
    print(f'\nModel RMSE for {target}: {rmse:.2f}')
    
    
    scores = cross_val_score(xgb, X_train, y_train, cv=5)
    print("Mean cross-validation score: %.2f" % scores.mean())

    kfold = KFold(n_splits=10, shuffle=True)
    kf_cv_scores = cross_val_score(xgb, X_train, y_train, cv=kfold )
    print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

    y_pred = xgb.predict(X_test)
    cm = confusion_matrix(y_test,y_pred)
    print(cm)
#     print(f'Model NRMSE for {target}: {rmse/(predictions.max()-predictions.min()):.2f}\n')
    
    return
    
    

In [5]:
run_model(df, 'winner', features=features)


Model score for training data for winner: 1.0000
Model score for testing data for winner: 1.0000

Model RMSE for winner: 0.00
Mean cross-validation score: 1.00
K-fold CV average score: 1.00
[[991   0]
 [  0 968]]
